In [42]:
%pwd


'd:\\Desktop\\MLOPS\\ds_wine_prediction'

In [43]:
import os
os.chdir("../")
%pwd

'd:\\Desktop\\MLOPS'

In [44]:
cd ds_wine_prediction

d:\Desktop\MLOPS\ds_wine_prediction


d:\Desktop\MLOPS\ds_wine_prediction\venv\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [45]:
%pwd

'd:\\Desktop\\MLOPS\\ds_wine_prediction'

In [46]:
%pwd

'd:\\Desktop\\MLOPS\\ds_wine_prediction'

In [47]:
import os
import sys
from pathlib import Path
from typing import List
import pandas as pd

In [48]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
   root_dir: Path
   train_data_path: Path
   test_data_path: Path
   model_name : str
   alpha: float
   l1_ratio: float
   target_column: str

In [49]:
from src.datascience.constants import *
from src.datascience.constants import CONFIG_FILE_PATH
from src.datascience.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
        
        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema 
        )
        return model_trainer_config
        

In [50]:

import os
import urllib.request as request
from src.datascience import logger
from sklearn.linear_model import ElasticNet
import joblib


In [61]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data=pd.read_csv(self.config.train_data_path)
        test_data=pd.read_csv(self.config.test_data_path)

        train_x=train_data.drop([self.config.target_column],axis=1)
        test_x=test_data.drop([self.config.target_column],axis=1)
        train_y=train_data[[self.config.target_column]]
        test_y=test_data[[self.config.target_column]]

        lr=ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio,random_state=42)
        lr.fit(train_x,train_y)

        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))




In [62]:
try:
    config=ConfigurationManager()
    model_trainer_config=config.get_model_trainer_config()
    model_trainer=ModelTrainer(config=model_trainer_config)
    model_trainer.train()
  
except Exception as e:
    raise e    

[2025-09-24 14:04:22,740: INFO: common: YAML file loaded successfully: D:\Desktop\MLOPS\ds_wine_prediction\config\config.yaml]
[2025-09-24 14:04:22,744: INFO: common: YAML file loaded successfully: D:\Desktop\MLOPS\ds_wine_prediction\params.yaml]
[2025-09-24 14:04:22,756: INFO: common: YAML file loaded successfully: D:\Desktop\MLOPS\ds_wine_prediction\schema.yaml]
[2025-09-24 14:04:22,759: INFO: common: Directory created or already exists: artifacts]
[2025-09-24 14:04:22,765: INFO: common: Directory created or already exists: artifacts/model_trainer]
